In [1]:
from google.cloud import bigquery

In [15]:
import plotly.express as px

In [12]:
from find_drives import *

In [2]:
bqc = bigquery.Client(project='spurs-sp2018')

In [24]:
shots_sql = """--sql
SELECT x, y, concat(pl.firstName, ' ', pl.lastName) as player_name, outcome, season
from ss.shots
join ss.players as pl on pl.id = shots.shooter_id
where concat(pl.firstName, ' ', pl.lastName) = 'Stephen Curry'
and season = '2018'
"""

In [25]:
df = bqc.query(shots_sql).to_dataframe()

In [26]:
df

,x,y,player_name,outcome,season
0,-24.07,-7.55,Stephen Curry,True,2018
1,-14.88,0.63,Stephen Curry,False,2018
2,-17.85,-9.41,Stephen Curry,False,2018
3,-14.32,-3.15,Stephen Curry,False,2018
4,-39.94,-6.06,Stephen Curry,False,2018
...,...,...,...,...,...
1685,-19.73,-15.50,Stephen Curry,True,2018
1686,-39.50,-22.89,Stephen Curry,True,2018
1687,-39.69,23.05,Stephen Curry,True,2018
1688,-22.17,16.47,Stephen Curry,True,2018


In [28]:
def get_shots_sql(player_name, season, x_range, y_range):
    return f"""--sql
    SELECT x, y, concat(pl.firstName, ' ', pl.lastName) as player_name, outcome, season
    from ss.shots
    join ss.players as pl on pl.id = shots.shooter_id
    where concat(pl.firstName, ' ', pl.lastName) = '{player_name}'
    and season = '{season}'
    and x between {x_range[0]} and {x_range[1]}
    and y between {y_range[0]} and {y_range[1]}
    """

In [29]:
def get_shots_dataframe(player_name, season, x_range, y_range):
    shots_sql = get_shots_sql(player_name, season, x_range, y_range)
    return bqc.query(shots_sql).to_dataframe()

In [38]:
def get_shots_fig(player_name, season, x_range, y_range):
    df = get_shots_dataframe(player_name, season, x_range, y_range)
    fg_pct = df['outcome'].sum() / len(df)
    fig = px.scatter(df, x='x', y='y', color='outcome', hover_data=['player_name', 'season'])
    # set fig title
    fig.update_layout(title=f'{player_name} {season} FG%: {fg_pct:.2f}')

    fig.update_traces(marker_size=3)
    draw_basketball_court(fig)
    return fig

In [45]:
args = {
    'player_name': 'Stephen Curry',
    'season': '2018',
    'x_range': [-44, -35],
    'y_range': [-5, 5]
}

In [46]:
fig = get_shots_fig(**args)
fig.show()